In [3]:
!pip install split-folders

In [4]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import transforms
import os
from PIL import Image
import torch
import ssl
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import splitfolders
import csv
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import warnings

#warnings.filterwarnings('ignore')


def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == '__main__':
    # Define the device to be used for training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set up the transform to resize and normalize the images
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]),
    ])

    # Update input folder and output folder paths
    input_folder = r"/kaggle/input/base-dataset-without-preprocessing/Without_any_preprocessing_small"
    output_folder = r"/kaggle/output/Images_no_preprocessing"

    ### Uncomment only for first time. once data is splitted into train and validation, comment it out
    splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2), group_prefix=None)

    # Create datasets for the training and testing sets
    train_dataset = torchvision.datasets.ImageFolder(output_folder + '/train', transform=transform)
    val_dataset = torchvision.datasets.ImageFolder(output_folder + '/val', transform=transform)
    train_size = len(train_dataset)
    val_size = len(val_dataset)

    # Create the data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True,num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=5, shuffle=True,num_workers=4)
    list_of_classes = os.listdir(r"/kaggle/output/Images_no_preprocessing/train")
    print(list_of_classes)
    classes = list(train_dataset.class_to_idx.keys())
    classes.sort()


    # Define the VGG model
    model = torchvision.models.vgg16(pretrained=True)
    num_features = model.classifier[0].in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_features, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, len(list_of_classes))
    )

    ## uncomment for CPU
    model = model.to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    START_LR = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=START_LR)
    model = model.to(device)
    criterion = criterion.to(device)

    ### uncomment for GPU
    model.cuda()
        # Train the model
    for epoch in range(10):
        ### uncomment for GPU
        torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch + 1, 10))
        print('-' * 10)

        running_loss = 0
        running_corrects = 0

        model.train()
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('training-classification-epoch' + str(epoch + 1) + '.csv')
        matrix = confusion_matrix(true_labels, predictions)
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-train-epoch' + str(epoch + 1) + '.csv')
        acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        print("accuracy for all classes in train phase", ACC)
        print("recall for all classes in train phase", TPR)
        print("precision for all classes in train phase", PPV)
        
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-train-epoch' + str(epoch + 1) + '.csv')
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
        
        # Validation phase
        running_loss = 0
        running_corrects = 0
        model.eval()  # set the model to evaluation mode
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
       
        #classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('val-classification-epoch' + str(epoch + 1) + '.csv')
        matrix = confusion_matrix(true_labels, predictions)
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-val-epoch' + str(epoch + 1) + '.csv')
        acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-val-epoch' + str(epoch + 1) + '.csv')
        print("accuracy for all classes in validation phase", ACC)
        print("recall for all classes in validation phase", TPR)
        print("precision for all classes in validation phase", PPV)
        print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
                
        

    # Save the model
    torch.save(model, 'vgg16_model.pth')




Copying files: 2225 files [00:12, 182.74 files/s]
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


['chtapa3', 'pygbat1', 'chibat1', 'combul2', 'gobbun1', 'grbcam1', 'abhori1', 'palfly2', 'whctur2', 'spwlap1', 'chespa1', 'gobwea1', 'norpuf1', 'rebfir2', 'whbtit5', 'slbgre1', 'colsun2', 'vilwea1', 'grewoo2', 'yebsto1', 'whbcan1', 'brican1', 'nubwoo1', 'carcha1', 'afghor1', 'spewea1', 'reccuc1', 'somgre1', 'afrjac1', 'blakit1']


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 80.5MB/s] 


Epoch 1/10
----------


100%|██████████| 354/354 [00:37<00:00,  9.32it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel

...

accuracy for all classes in validation phase [0.96724891 0.98034934 0.98471616 0.94978166 0.98471616 0.95633188
 0.99563319 0.97816594 0.99126638 0.96724891 0.91266376 0.98253275
 0.99781659 0.9650655  0.94323144 0.99126638 0.98253275 0.99344978
 0.99126638 0.91484716 0.98253275 0.98908297 0.95851528 0.98034934
 0.96069869 0.95196507 0.99563319 0.99126638 1.         0.99563319]
recall for all classes in validation phase [0.69230769 0.6        0.5        0.66037736 0.         0.4516129
 0.         0.8125     0.33333333 0.7027027  0.83050847 0.82352941
 0.         0.31578947 0.47619048 0.         0.2        0.
 0.33333333 0.8        0.68       0.2        0.875      0.14285714
 0.75       0.44444444 0.         0.2        1.         0.        ]
precision for all classes in validation phase [0.72       0.75       0.42857143 0.875      0.         0.82352941
        nan 0.65       1.         0.86666667 0.62025316 0.73684211
        nan 0.66666667 0.4               nan 0.2               nan
 0

100%|██████████| 354/354 [00:32<00:00, 10.84it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel

accuracy for all classes in train phase [0.98925339 0.98812217 0.9949095  0.98925339 0.99264706 0.98585973
 0.99773756 0.98076923 0.99377828 0.99038462 0.96210407 0.99038462
 0.99773756 0.97115385 0.98472851 0.99151584 0.99264706 0.99547511
 0.99717195 0.98755656 0.99830317 0.98755656 0.99038462 0.98925339
 0.99434389 0.97171946 0.99717195 0.98755656 1.         0.99717195]
recall for all classes in train phase [0.92       0.8245614  0.75       0.96650718 0.69565217 0.90163934
 0.         0.71428571 0.66666667 0.95833333 0.91025641 0.90625
 0.         0.65333333 0.91463415 0.1875     0.47058824 0.36363636
 0.81818182 0.64864865 0.97938144 0.22222222 0.93710692 0.4
 0.89361702 0.75714286 0.5        0.27777778 1.         0.        ]
precision for all classes in train phase [0.89320388 0.81034483 0.85714286 0.94392523 0.72727273 0.89430894
        nan 0.73770492 0.77777778 0.9261745  0.82239382 0.84057971
        nan 0.66216216 0.78947368 0.6        0.66666667 0.8
 0.75       0.72727273 0.

100%|██████████| 92/92 [00:02<00:00, 31.20it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_4

accuracy for all classes in validation phase [0.98034934 0.97816594 0.98689956 0.94104803 0.98034934 0.930131
 0.99563319 0.98471616 0.99126638 0.96069869 0.93231441 0.96943231
 0.99781659 0.96069869 0.96943231 0.99126638 0.98253275 0.99344978
 0.99344978 0.98034934 0.98689956 0.98471616 0.95196507 0.96943231
 0.98471616 0.94759825 0.99563319 0.98471616 1.         0.99563319]
recall for all classes in validation phase [0.88461538 0.46666667 0.66666667 0.56603774 0.83333333 0.67741935
 0.         0.5625     0.33333333 0.75675676 0.71186441 0.88235294
 0.         0.57894737 0.66666667 0.         0.         0.
 0.33333333 0.4        0.88       0.2        0.85       0.42857143
 0.58333333 0.66666667 0.         0.2        1.         0.        ]
precision for all classes in validation phase [0.79310345 0.77777778 0.5        0.88235294 0.38461538 0.48837209
        nan 1.         1.         0.75675676 0.75       0.55555556
        nan 0.52380952 0.66666667        nan 0.                nan
 0.